In [0]:
from pyspark.errors import IllegalArgumentException
import json
import os

In [0]:
dbutils.widgets.text('target_catalog', 'solution_acc')
dbutils.widgets.text('target_schema', 'energy_forecasting')
dbutils.widgets.text('secret_scope', 'stuart')
dbutils.widgets.text('secret_key', 'cds-token')
dbutils.widgets.text('ignore_errors', 'false')
dbutils.widgets.text('clone_raw_data', 'false')
dbutils.widgets.text('overwrite_data', 'false')

_ignore_errors = dbutils.widgets.get('ignore_errors').lower() == 'true'
target_catalog = dbutils.widgets.get('target_catalog')
target_schema = dbutils.widgets.get('target_schema')
secret_scope = dbutils.widgets.get('secret_scope')
secret_key = dbutils.widgets.get('secret_key')
clone_raw_data = dbutils.widgets.get('clone_raw_data').lower() == 'true'
overwrite_data = dbutils.widgets.get('overwrite_data').lower() == 'true'

In [0]:
if target_catalog not in spark.catalog.listCatalogs():
  print('Target catalog not found, attempting to create')
  try: 
    spark.sql(f"CREATE CATALOG IF NOT EXISTS {target_catalog}")
  except Exception as E:
    print(f'Exception creating catalog, {_ignore_errors=}')
    if not _ignore_errors:
      raise E


if not spark.catalog.databaseExists(f"{target_catalog}.{target_schema}"):
  spark.sql(f"CREATE SCHEMA IF NOT EXISTS {target_catalog}.{target_schema}")

print('Creating managed volume in newly created schema')
spark.sql(f"CREATE VOLUME IF NOT EXISTS {target_catalog}.{target_schema}.unstructured_data")
try:
   secret_exists = dbutils.secrets.get(secret_scope, secret_key)
except IllegalArgumentException as E:
  print("Missing secret scope or key, please ensure they exist or you have access to them.")
  raise E

In [0]:
if 'config.json' in os.listdir('./includes/'):
  print("config file found, will overwrite with current run values")

with open('./includes/config.json', 'w') as f:
  f.write(json.dumps({
    'target_catalog': target_catalog,
    'target_schema': target_schema,
    'secret_scope': secret_scope,
    'secret_key': secret_key,
    'clone_raw_data': clone_raw_data,
    'overwrite_data': overwrite_data
  }))
